In [582]:
from gensim import corpora, models, similarities, utils
import numpy as np
import pandas as pd
import nltk
import cPickle as pickle
import re

In [587]:
NWORDS_SHOWN = 15# Number of words per topic to extract
SELECTED = [1,2,4,8,11,14,16,25]#Numbers of the topics that I will include in the table

lda = pickle.load(open('lda_ortho_30t.p','rb'))

In [588]:
# texts = corpora.Dictionary.load('orthopedists.dict')
# print texts
#corpus = [dictionary.doc2bow(text) for text in texts]

In [589]:
def get_parts_of_speech(sel_topic, temp_df):
    """Get Parts of Speech for that selected topic's words > pos DF
    Arguments:
    sel_topic = the integer number corresponding to one of the T topics in the LDA
    temp_df is a DataFrame holding all the words highly associated with that topic in a row
    Returns: pos_tup = tuples with each word number and corresponding POS code (see this PDF for code mapping)
    # https://blogs.princeton.edu/etc/files/2014/03/Text-Analysis-with-NLTK-Cheatsheet.pdf"""
    
    words = []
    words = temp_df.loc[0,:]#extract words from the dataframe
    pos_tup = nltk.pos_tag(words)# This tags the words (list) with parts of speech codes
    
#     # for debugging.  Allows you to print for a given topic
#     if sel_topic==1:
#         pos = pd.DataFrame([t for t in pos_tup]).transpose()
#         print sel_topic, pos,'/n'
        
    return pos_tup

In [590]:
def filter_POS(df, pos_tup):
    """Receives a dataframe. Outputs a filtered dict in which each word is mapped onto the parts of speech.
    Words are excluded from the dict if they have one of the POS considered less informative."""
    # Remove less informative parts of speech from the topics, so word
    # RB = AVB; CD = Cardinal Number; PRP/PRP$ = personal pronoun; RB=ADVERB; IN = Preposition
    
    #print pos_tup,'\n'
    
    # dict of word number, POS tuples
    table_dict={}
    
    for c in df.columns:
        if pos_tup[c][1]!='CD' and pos_tup[c][1]!='PRP' and pos_tup[c][1]!='IN' and pos_tup[c][1]!='RB' and pos_tup[c][1]!='CC':
        #if pos_tup[c][1]== [stop for stop in pow_stop]:
            table_dict[c+1]=df[c][0]#set key +1 because topics start at 1 not 0
            
    #pow_stop = ['CD','PRP','IN','RB']#tried to clean if statement with loop didn't work
    return table_dict

In [594]:
# CREATE FINAL DF of FILTERED TOP-WORD MAPPINGS FOR TABLE ON WEBSITE
# 
# Pull words lists for selected topics - output for website

#Initialize
topic_word_df=pd.DataFrame()#unfiltered by POS
Ftopic_word_df=pd.DataFrame()#filtered by POS
frames = []# list of "topic-word map" dataframes for each topic

#temp_df is a df containing words for a single topic, which gets appended to final df, and reused for each loop cycle

for sel_topic in SELECTED:
    topic = lda.show_topic(sel_topic, NWORDS_SHOWN)#type list
    temp_df=pd.DataFrame([(tup[1].encode("utf-8").replace(',','')) for tup in topic]).transpose()#got strange error trying to make transpose a separate line of code... 
    
    # Tag Parts of speech for each word, and filter/remove words with less useful POS
    pos_tup = get_parts_of_speech(sel_topic, temp_df)#tup (word, POS)
    table_dict = filter_POS(temp_df, pos_tup)#dict key:value --> word#:word
    
    # Add the Topic # in a column at the beginning, and append to master DF
    #temp_df.insert(0,'Topic',sel_topic)
    topic_word_df = topic_word_df.append([temp_df],ignore_index=True)
    
    # STILL DEBUGGING
    word_indices = [i for i in table_dict.keys()]
    words_filtered = [t for t in table_dict.values()]
    temp_filtered_df = pd.DataFrame([words_filtered],columns=word_indices)
    #topics_filtered_df = topics_filtered_df.transpose()
    #topics_filtered_df.index = np.arange(0,topics_filtered_df.shape[0])
    #temp_filtered_df.insert(0,'Topic',sel_topic)
    Ftopic_word_df = Ftopic_word_df.append([temp_filtered_df],ignore_index=True)
    
    #row_idx = selected.index(sel_topic)
    #frames.insert(row_idx, topics_filtered_df)

#Ftopic_word_df = pd.concat(frames)
topic_word_df.insert(0,'Topic',SELECTED)
Ftopic_word_df.insert(0,'Topic',SELECTED)
#topic_word_df.ix[7,3]='image'#replace a name with a later word in this 

In [595]:
topic_word_df.head(10)

,Topic,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,knees,knee,pain,hip,surgeries,orthopedic,one,surgeons,great,&,patellas,time,left,i've,life
1,2,hip,pain,knee,time,right,back,i'm,care,good,guy,labrum,day,since,hips,left
2,4,time,really,care,like,best,staff,treated,office,shoulder,good,appointment,orthopedic,always,patient,great
3,8,knee,pain,much,time,years,running,orthopedic,saw,work,good,many,without,best,even,team
4,11,appointment,minutes,time,staff,waiting,day,told,one,me,late,first,say,luke,office,mri
5,14,best,time,he's,good,bedside,physical,weeks,pain,knee,one,months,recommend,even,amazing,care
6,16,great,one,time,even,really,never,office,every,experience,knee,staff,right,i'm,christina,i've
7,25,mri,office,needed,atkin,us,one,back,physical,medical,images,new,really,day,hip,tried


In [593]:
Ftopic_word_df.head(10)

,Topic,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1,knees,knee,pain,hip,surgeries,orthopedic,NaN,surgeons,great,NaN,patellas,time,left,i've,life
1,2,hip,pain,knee,time,NaN,NaN,i'm,care,good,guy,labrum,day,NaN,hips,left
2,4,time,NaN,care,NaN,best,staff,treated,office,shoulder,good,appointment,orthopedic,always,patient,great
3,8,knee,pain,NaN,time,years,running,orthopedic,saw,work,good,many,NaN,best,NaN,team
4,11,appointment,minutes,time,staff,waiting,day,told,NaN,NaN,late,first,say,luke,office,mri
5,14,best,time,he's,good,bedside,physical,weeks,pain,knee,NaN,months,recommend,NaN,amazing,care
6,16,great,NaN,time,NaN,NaN,NaN,office,every,experience,knee,staff,NaN,i'm,christina,i've
7,25,mri,office,needed,atkin,NaN,NaN,NaN,physical,medical,images,new,NaN,day,hip,tried


In [489]:
# save word dfs & dict

pickle_out = open('table_dict.p', 'wb')
pickle.dump(table_dict, pickle_out)

pickle_out = open('topic_word_df.p', 'wb')
pickle.dump(topic_word_df, pickle_out)

pickle_out = open('Ftopic_word_df.p', 'wb')
pickle.dump(Ftopic_word_df, pickle_out)

In [320]:
import csv

def write_csv(filename, words):
    with open(filename,'w',newline='') as csvfile:
        wr = csv.writer(csvfile, delimiter=',',
                        quotechar='|',quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
        #wr.writerow(table_dict.keys())
        wr.writerows(words) 
    print('\nCSV file saved.\n')
    
write_csv('table_dict.csv', mywords)

TypeError: 'newline' is an invalid keyword argument for this function

In [540]:
a = np.array([[0,1,2],[3,4,5]])
print a
temp = pd.DataFrame(a)
print '\nDataframe = \n',temp

temp.index = [3,4]
print '\nDataframe = \n',temp

[[0 1 2]
 [3 4 5]]

Dataframe = 
   0  1  2
0  0  1  2
1  3  4  5

Dataframe = 
   0  1  2
3  0  1  2
4  3  4  5
